In [2]:
import pandas as pd 
df_train_prev_loans=pd.read_csv('Datasets/trainprevloans.csv')
df_train_prev_loans.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,closeddate,referredby,firstduedate,firstrepaiddate
0,8a2a81a74ce8c05d014cfb32a0da1049,301682320,2,2016-08-15 18:22:40.000000,2016-08-15 17:22:32.000000,10000.0,13000.0,30,2016-09-01 16:06:48.000000,NaN,2016-09-14 00:00:00.000000,2016-09-01 15:51:43.000000
1,8a2a81a74ce8c05d014cfb32a0da1049,301883808,9,2017-04-28 18:39:07.000000,2017-04-28 17:38:53.000000,10000.0,13000.0,30,2017-05-28 14:44:49.000000,NaN,2017-05-30 00:00:00.000000,2017-05-26 00:00:00.000000
2,8a2a81a74ce8c05d014cfb32a0da1049,301831714,8,2017-03-05 10:56:25.000000,2017-03-05 09:56:19.000000,20000.0,23800.0,30,2017-04-26 22:18:56.000000,NaN,2017-04-04 00:00:00.000000,2017-04-26 22:03:47.000000
3,8a8588f35438fe12015444567666018e,301861541,5,2017-04-09 18:25:55.000000,2017-04-09 17:25:42.000000,10000.0,11500.0,15,2017-04-24 01:35:52.000000,NaN,2017-04-24 00:00:00.000000,2017-04-24 00:48:43.000000
4,8a85890754145ace015429211b513e16,301941754,2,2017-06-17 09:29:57.000000,2017-06-17 08:29:50.000000,10000.0,11500.0,15,2017-07-14 21:18:43.000000,NaN,2017-07-03 00:00:00.000000,2017-07-14 21:08:35.000000


## Attributes 
<li> Customerid (Primary key used to merge to other data) </li> 
<li> Systemloanid (The id associated with the particular loan. The same customerId can have multiple </li> 
<li> Systemloanid’s for each loan he/she has taken out)</li> 
<li> Loannumber (The number of the loan that you have to predict)</li> 
<li> Approveddate (Date that loan was approved)</li> 
<li> Creationdate (Date that loan application was created)</li> 
<li> Loanamount (Loan value taken)</li> 
<li> Totaldue (Total repayment required to settle the loan - this is the capital loan value disbursed +interest and fees) </li> 
<li> Termdays (Term of loan) </li> 
<li> Referredby (customerId of the customer that referred this person - is missing, then not referred)</li> 
<li> Good_bad_flag (good = settled loan on time; bad = did not settled loan on time) - this is the target variable that we need to predict</li> 

### We need to comeup with a summarised position for the customer previous loans 

In [24]:
df_train_prev_loans_summ=df_train_prev_loans.groupby('customerid',as_index=False).agg({'systemloanid':'nunique',
                                                                          'loanamount':'sum',
                                                                          'totaldue':'sum',
                                                                          'termdays':'mean',
                                                                          'referredby':pd.Series.mode,
                                                                          'approveddate':'max',
                                                                          'creationdate':'max',
                                                                          'closeddate':'max',
                                                                          'firstduedate':'max',
                                                                          'firstrepaiddate':'max'})

In [25]:
df_train_prev_loans_summ.head()

,customerid,systemloanid,loanamount,totaldue,termdays,referredby,approveddate,creationdate,closeddate,firstduedate,firstrepaiddate
0,8a1088a0484472eb01484669e3ce4e0b,1,10000.0,11500.0,15.000000,[],2017-07-02 19:19:01.000000,2017-07-02 18:17:53.000000,2017-07-24 20:50:43.000000,2017-07-18 00:00:00.000000,2017-07-24 20:40:34.000000
1,8a1a1e7e4f707f8b014f797718316cad,4,70000.0,89500.0,37.500000,[],2017-06-21 22:09:51.000000,2017-06-21 21:09:44.000000,2017-07-26 19:48:11.000000,2017-07-21 00:00:00.000000,2017-07-20 15:05:33.000000
2,8a1a32fc49b632520149c3b8fdf85139,7,90000.0,106500.0,19.285714,[],2017-06-13 11:02:36.000000,2017-06-13 10:01:27.000000,2017-07-13 18:45:26.000000,2017-07-13 00:00:00.000000,2017-07-13 18:35:18.000000
3,8a1eb5ba49a682300149c3c068b806c7,8,130000.0,162400.0,33.750000,[],2017-05-23 08:25:46.000000,2017-05-23 07:24:38.000000,2017-07-12 13:02:44.000000,2017-06-22 00:00:00.000000,2017-06-30 21:03:22.000000
4,8a1edbf14734127f0147356fdb1b1eb2,2,20000.0,24500.0,22.500000,[],2017-06-05 17:01:03.000000,2017-06-05 16:00:57.000000,2017-06-27 18:13:52.000000,2017-07-05 00:00:00.000000,2017-06-27 18:03:43.000000


In [27]:
df_train_prev_loans_summ.rename(columns={"systemloanid":"No_of_prevloans","loanamount":"Total_amount_borrowed",
                                         "totaldue":"Total_amount_due","termdays":"Average_termdays","referredby":"most_referredby",
                                         "approveddate":"last_approveddate","creationdate":"last_creationdate","closeddate":"last_closeddate",
                                         "firstduedate":"last_firstduedate","firstrepaiddate":"last_firstrepaiddate"},inplace=True)
df_train_prev_loans_summ.head()

,customerid,No_of_prevloans,Total_amount_borrowed,Total_amount_due,Average_termdays,most_referredby,last_approveddate,last_creationdate,last_closeddate,last_firstduedate,last_firstrepaiddate
0,8a1088a0484472eb01484669e3ce4e0b,1,10000.0,11500.0,15.000000,[],2017-07-02 19:19:01.000000,2017-07-02 18:17:53.000000,2017-07-24 20:50:43.000000,2017-07-18 00:00:00.000000,2017-07-24 20:40:34.000000
1,8a1a1e7e4f707f8b014f797718316cad,4,70000.0,89500.0,37.500000,[],2017-06-21 22:09:51.000000,2017-06-21 21:09:44.000000,2017-07-26 19:48:11.000000,2017-07-21 00:00:00.000000,2017-07-20 15:05:33.000000
2,8a1a32fc49b632520149c3b8fdf85139,7,90000.0,106500.0,19.285714,[],2017-06-13 11:02:36.000000,2017-06-13 10:01:27.000000,2017-07-13 18:45:26.000000,2017-07-13 00:00:00.000000,2017-07-13 18:35:18.000000
3,8a1eb5ba49a682300149c3c068b806c7,8,130000.0,162400.0,33.750000,[],2017-05-23 08:25:46.000000,2017-05-23 07:24:38.000000,2017-07-12 13:02:44.000000,2017-06-22 00:00:00.000000,2017-06-30 21:03:22.000000
4,8a1edbf14734127f0147356fdb1b1eb2,2,20000.0,24500.0,22.500000,[],2017-06-05 17:01:03.000000,2017-06-05 16:00:57.000000,2017-06-27 18:13:52.000000,2017-07-05 00:00:00.000000,2017-06-27 18:03:43.000000


In [3]:
df_train_demographics=pd.read_csv('Datasets/traindemographics.csv')
df_train_demographics.head()

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN,NaN
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,NaN,Permanent,NaN
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN
3,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,NaN,Permanent,NaN
4,8a858e785acd3412015acd48f4920d04,1982-11-22 00:00:00.000000,Savings,8.455332,11.971410,GT Bank,NaN,Permanent,NaN


In [5]:
# checking unique customers 
df_train_demographics.customerid.nunique()

4334

In [11]:
df_train_demographics.count()

customerid                    4346
birthdate                     4346
bank_account_type             4346
longitude_gps                 4346
latitude_gps                  4346
bank_name_clients             4346
bank_branch_clients             51
employment_status_clients     3698
level_of_education_clients     587
dtype: int64

### since the record count for customerid is higher than the uniques ids then there must be duplicates 

In [12]:
len(df_train_demographics)

4346

In [13]:
#dropping duplicates 
df_train_demographics=df_train_demographics.drop_duplicates(subset='customerid', keep='first', inplace=False)
len(df_train_demographics)

4334

## Attributes
<li> Customerid (Primary key used to merge to other data)</li>
<li> Birthdate (date of birth of the customer)
<li> Bank_account_type (type of primary bank account)</li>
<li> Longitude_gps</li>
<li> Latitude_gps</li>
<li> Bank_name_clients (name of the bank)</li>
<li> Bank_branch_clients (location of the branch - not compulsory - so missing in a lot of the cases)</li>
<li> Employment_status_clients (type of employment that customer has)
<li> Level_of_education_clients (highest level of education</li>

In [14]:
df_train_perf=pd.read_csv('Datasets/trainperf.csv')
df_train_perf.head()

,customerid,systemloanid,loannumber,approveddate,creationdate,loanamount,totaldue,termdays,referredby,good_bad_flag
0,8a2a81a74ce8c05d014cfb32a0da1049,301994762,12,2017-07-25 08:22:56.000000,2017-07-25 07:22:47.000000,30000.0,34500.0,30,NaN,Good
1,8a85886e54beabf90154c0a29ae757c0,301965204,2,2017-07-05 17:04:41.000000,2017-07-05 16:04:18.000000,15000.0,17250.0,30,NaN,Good
2,8a8588f35438fe12015444567666018e,301966580,7,2017-07-06 14:52:57.000000,2017-07-06 13:52:51.000000,20000.0,22250.0,15,NaN,Good
3,8a85890754145ace015429211b513e16,301999343,3,2017-07-27 19:00:41.000000,2017-07-27 18:00:35.000000,10000.0,11500.0,15,NaN,Good
4,8a858970548359cc0154883481981866,301962360,9,2017-07-03 23:42:45.000000,2017-07-03 22:42:39.000000,40000.0,44000.0,30,NaN,Good


## Attributes 
<li> Customerid (Primary key used to merge to other data)</li>
<li> Systemloanid (The id associated with the particular loan. The same customerId can have multiple </li>
<li> Systemloanid’s for each loan he/she has taken out)</li>
<li> Loannumber (The number of the loan that you have to predict)</li>
<li> Approveddate (Date that loan was approved)
<li> Creationdate (Date that loan application was created)</li>
<li> Loanamount (Date that loan application was created)
<li> Totaldue (Total repayment required to settle the loan - this is the capital loan value disbursed +interest and fees) termdays (Term of loan)</li>
<li> Closeddate (Date that the loan was settled)</li>
<li> Referredby (customerId of the customer that referred this person - is missing, then not refrerred)</li>
<li> Firstduedate (Date of first payment due in cases where the term is longer than 30 days. So in the case where the term is 60+ days - then there are multiple monthly payments due - and this dates reflects The date of the first payment)</li>
<li> firstrepaiddate (Actual date that he/she paid the first payment as defined above)</li>